In [1]:
import json 
import pandas as pd 

# Test model results 

In [2]:
test_model_results = []

with open("test_responses.jsonl",'r') as f : 
    for line in f: 
        if line.strip():
            data = json.loads(line)
            test_model_results.append(data)


In [3]:
df_test_model = pd.DataFrame(test_model_results)
df_test_model

,test model provider,test model version,Encountered Problems,test model response,sample_id
0,openai,gpt-3.5-turbo,False,Based on the information provided in the conte...,0
1,openai,gpt-3.5-turbo,False,1. Wüsthof - 14 degrees\n2. MAC - 15 degrees\n...,1


In [4]:
print(df_test_model['test model response'].iloc[0])

Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation. 

Cognitive engagement is important because it is thought to support the development of a "cognitive reserve," which may protect against brain cell damage caused by dementia. Engaging in activities that challenge and stimulate the brain throughout life may help reduce the risk of developing dementia.


In [6]:
print(df_test_model['test model response'].iloc[1])

1. Wüsthof - 14 degrees
2. MAC - 15 degrees
3. Messermeister - 15 degrees
4. Cangshan - 16 degrees
5. Shun and Kai - 16 degrees
6. Chroma - 10-20 degrees
7. F. DICK (Friedr. DICK) - 15-20 degrees
8. Fischer-Bargoin - 15-20 degrees
9. Global - 10-15 degrees
10. Korin - 10-20 degrees
11. Victorinox - 30-40 degrees (15-20 degrees on each side)
12. Zwilling J. A. Henckels and Miyabi - 15 degrees (9-12 degrees for certain knives)


# Judge 

In [7]:
judges_result = []

with open("judge_responses.jsonl",'r') as f : 
    for line in f:
        if line.strip():
            content = json.loads(line)
            judges_result.append(content)

In [8]:
df_judges = pd.DataFrame(judges_result)
df_judges

,judge model provider,judge model version,Encountered Problems,judge model response,judgeded_model,sample_id,verdict
0,openai,gpt-4o,False,"{""sentence"": ""Based on the information provide...",openai:gpt-3.5-turbo,0,Inaccurate
1,openai,gpt-4o,False,"{""sentence"": ""1. Wüsthof - 14 degrees"", ""label...",openai:gpt-3.5-turbo,1,Inaccurate


In [9]:
print(len(df_test_model))
print(len(df_judges))

2
2


In [11]:
print(df_judges['judge model response'].iloc[0])

{"sentence": "Based on the information provided in the context document, your risk factors for dementia include high blood pressure, diabetes, obesity, lack of physical activity, poor diet, high alcohol consumption, low cognitive engagement, depression, traumatic brain injury, hearing loss, and social isolation.", "label": "supported", "rationale": "The context lists these factors as increasing the risk of developing dementia.", "excerpt": "high blood pressure...diabetes...Obesity...lack of physical activity...poor diet...high alcohol consumption...low cognitive engagement...depression...traumatic brain injury...hearing loss...social isolation."}
{"sentence": "Cognitive engagement is important because it is thought to support the development of a \"cognitive reserve,\" which may protect against brain cell damage caused by dementia.", "label": "supported", "rationale": "The context explains that cognitive engagement supports the development of a cognitive reserve, which may protect agai

In [12]:
print(df_judges['judge model response'].iloc[1])

{"sentence": "1. Wüsthof - 14 degrees", "label": "supported", "rationale": "The context states that the sharpening angle for standard Wüsthof blades is 14 degrees.", "excerpt": "The sharpening angle for standard blades is 14 degrees, and for Asian-style blades (Santokus, Nakiris, Chai Daos) it’s 10 degrees."}
{"sentence": "2. MAC - 15 degrees", "label": "supported", "rationale": "The context mentions that MAC knives have factory edges of 15 degrees.", "excerpt": "MAC knives have factory edges of 15 degrees."}
{"sentence": "3. Messermeister - 15 degrees", "label": "supported", "rationale": "The context states that Messermeister Elité and Park Plaza knives have a 15-degree angle.", "excerpt": "Messermeister Elité and Park Plaza knives have a 15-degree angle."}
{"sentence": "4. Cangshan - 16 degrees", "label": "supported", "rationale": "The context specifies that Cangshan knives are sharpened to a 16-degree edge.", "excerpt": "Cangshan knives are sharpened to an Asian-style 16-degree edge